In [ ]:
import pandas as pd
from client import RestClient

In [ ]:
client = RestClient("tegnuyofyi@yevme.com", "1194a27214f232dd")
def get_keywords(keyword, depth = 4, location = 'United States'):
    post_data = dict()
    post_data[len(post_data)] = dict(
        keyword=keyword, 
        location_name=location,
        language_name="English",
        depth=depth, 
    )
    post_data2 = dict()
    post_data2[len(post_data2)] = dict(
        keywords=[keyword],
        location_name=location,
        language_name="English",
        depth=depth,
    )
    response = client.post("/v3/dataforseo_labs/related_keywords/live", post_data)
    response2 = client.post("/v3/dataforseo_labs/keyword_ideas/live", post_data2)
    response3 = client.post("/v3/dataforseo_labs/keyword_suggestions/live", post_data)
    
    return {
        'related' : response,
        'ideas' : response2, 
        'suggestions' : response3
    }


In [ ]:
def extract_keywords(responses):
    key_list = []
    sources = []
    
    if responses['related']["status_code"] == 20000 and not responses['related']['tasks'][0]['result'][0]['items'] == None:
        for x in range(len(responses['related']['tasks'][0]['result'][0]['items'])):
            res = responses['related']['tasks'][0]['result'][0]['items'][x]['related_keywords']
            if res is not None:
                key_list.extend(res)
        print(f"{len(key_list)} related")
        sources.extend(['related' for _ in range(len(key_list))])
        
    if responses['ideas']["status_code"] == 20000 and not responses['ideas']['tasks'][0]['result'][0]['items'] == None:
        # not good ideas
        print(f"{len(responses['ideas']['tasks'][0]['result'][0]['items'])} ideas")
        for x in range(len(responses['ideas']['tasks'][0]['result'][0]['items'])):
            res = responses['ideas']['tasks'][0]['result'][0]['items'][x]['keyword']
            if res is not None:
                key_list.append(res)
        sources.extend(['ideas' for _ in range(
            len(responses['ideas']['tasks'][0]['result'][0]['items']))])
                
    if responses['suggestions']["status_code"] == 20000 and not responses['suggestions']['tasks'][0]['result'][0]['items'] == None:
        print(f"{len(responses['suggestions']['tasks'][0]['result'][0]['items'])} suggestions")
        for x in range(len(responses['suggestions']['tasks'][0]['result'][0]['items'])):
            res = responses['suggestions']['tasks'][0]['result'][0]['items'][x]['keyword']
            if res is not None:
                key_list.append(res)
        sources.extend(['suggestions' for _ in range(
            len(responses['suggestions']['tasks'][0]['result'][0]['items']))])
    # else:
    #     print("error. Code: %d Message: %s" %
    #           (responses['all]["status_code"], responses['all']["status_message"]))
    temp = {
        'Keywords' : key_list,
        'Sources' : sources
    }
    df = pd.DataFrame.from_dict(temp)
    return df


In [ ]:
keyword_list = ['ARR Loans', 'Revenue Based Loans', 'Loans for SaaS Companies']

full_df = pd.DataFrame(columns=['Keywords', 'Keywords2', 'Sources'])
for keyword in keyword_list:
    print(f'keyword : {keyword}')
    keywords = get_keywords(keyword, depth=4)
    extracted = extract_keywords(keywords)
    keywords2 = [keyword for _ in range(extracted.shape[0])]
    extracted['Keywords2'] = keywords2
    full_df = pd.concat([full_df, extracted])

In [ ]:
full_df.to_csv('keywords_dataset.csv', index = False)